In [ ]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from dotenv import load_dotenv

# 환경 변수 로드 (OPENAI_API_KEY를 위해)
load_dotenv()

# --- ChromaDB 설정 ---
# 병합된 ChromaDB 파일(chroma.sqlite3)이 들어있는 디렉토리 경로를 정확히 지정하세요.
# 예시: r"C:\Users\mojih\OneDrive\바탕 화면\Project4\SKN13-4th-4Team\ai_influencer_agent\merged_rehabilitation_chroma_data"
# 이 경로를 당신의 실제 경로로 수정해야 합니다!
PERSIST_DIRECTORY = r"C:\Users\mojih\OneDrive\바탕 화면\Project4\SKN13-4th-4Team\ai_influencer_agent\merge"

# ChromaDB 컬렉션 이름 (이전에 데이터를 저장했던 컬렉션 이름과 동일해야 합니다)
COLLECTION_NAME = "rehabilitation_articles_openai"

# OpenAI 임베딩 모델 초기화 (DB 생성 시 사용한 모델과 동일해야 합니다)
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 저장된 Chroma 벡터 DB 불러오기
print(f"ChromaDB를 '{PERSIST_DIRECTORY}' 경로에서 불러오는 중...")
try:
    vectordb = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings_model,
        collection_name=COLLECTION_NAME # 명시적으로 컬렉션 이름을 지정하여 올바른 컬렉션을 로드
    )
    print("ChromaDB 불러오기 성공.")
except Exception as e:
    print(f"ChromaDB 불러오기 오류: {e}")
    print("경로가 올바른지, 또는 해당 디렉토리 안에 'chroma.sqlite3' 파일이 있는지 확인해주세요.")
    exit()

# 컬렉션에 문서가 몇 개 있는지 확인
try:
    doc_count = vectordb._collection.count()
    print(f"로드된 컬렉션 '{COLLECTION_NAME}'에 총 {doc_count}개의 문서가 있습니다.")
    if doc_count == 0:
        print("❗ 경고: 컬렉션에 문서가 없습니다. 병합이 제대로 되었는지 확인해주세요.")
except Exception as e:
    print(f"컬렉션 문서 수 확인 중 오류 발생: {e}")


# 관련 문서 최대 3개 검색하는 retriever 생성
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# --- Tool 생성 (선택 사항, RAG 에이전트에서 사용) ---
q_a_tool = create_retriever_tool(
    retriever,
    name="q_a_tool",
    description="사용자의 의학 관련 궁금증에 대해 쉽게 설명하는 도구입니다."
)

# --- 문서 검색 테스트 ---
print("\n--- 문서 검색 테스트 시작 ---")

# 임의의 질문을 입력하여 테스트합니다.
# 당신이 수집한 논문 내용과 관련된 질문일수록 더 좋은 결과를 얻을 수 있습니다.
test_query = "재활 운동이 무릎 통증에 어떤 영향을 미치나요?"
# test_query = "필라테스의 효과는 무엇인가요?"
# test_query = "생체역학 연구의 최신 동향은?"

print(f"질문: '{test_query}'")

try:
    # retriever를 사용하여 관련 문서 검색 (deprecated 경고 해결을 위해 invoke 사용)
    relevant_documents = retriever.invoke(test_query) # get_relevant_documents 대신 invoke 사용

    if relevant_documents:
        print(f"\n'{test_query}'에 대한 검색 결과 ({len(relevant_documents)}개 문서):")
        for i, doc in enumerate(relevant_documents):
            print(f"\n--- 문서 {i+1} ---")
            print(f"소스 (메타데이터): {doc.metadata.get('title', '제목 없음')}") # 메타데이터에서 'title' 사용
            print(f"내용 (일부): {doc.page_content[:500]}...") # 문서 내용의 앞부분 출력
            print("-" * 30)
    else:
        print("관련 문서를 찾을 수 없습니다.")
        print("팁: 질문을 좀 더 일반적이거나, 수집된 논문의 키워드와 유사하게 변경해 보세요.")

except Exception as e:
    print(f"문서 검색 중 오류 발생: {e}")
    print("OpenAI API 키가 유효한지, 또는 임베딩 모델 설정이 올바른지 확인해주세요.")

print("\n--- 문서 검색 테스트 완료 ---")



ChromaDB를 'C:\Users\mojih\OneDrive\바탕 화면\Project4\SKN13-4th-4Team\ai_influencer_agent\merged_rehabilitation_chroma_data' 경로에서 불러오는 중...
ChromaDB 불러오기 성공.
로드된 컬렉션 'rehabilitation_articles_openai'에 총 0개의 문서가 있습니다.
❗ 경고: 컬렉션에 문서가 없습니다. 병합이 제대로 되었는지 확인해주세요.

--- 문서 검색 테스트 시작 ---
질문: '재활 운동이 무릎 통증에 어떤 영향을 미치나요?'
관련 문서를 찾을 수 없습니다.
팁: 질문을 좀 더 일반적이거나, 수집된 논문의 키워드와 유사하게 변경해 보세요.

--- 문서 검색 테스트 완료 ---
